In [1]:
import numpy as np
import sys
import logging
#logging.basicConfig(level=logging.INFO)
import random
import tenpy
from tenpy.models.tf_ising import TFIChain
from tenpy.networks.purification_mps import PurificationMPS
from tenpy.algorithms.purification import PurificationTEBD, PurificationApplyMPO
import itertools

import scipy as sc
import scipy.sparse as sparse
import csv  
import datetime
import time
from scipy.sparse.linalg import expm_multiply
import concurrent.futures
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import sqrtm
import pandas as pdp
from scipy.sparse.linalg import eigsh

In [2]:
pauli_x = sparse.csr_matrix(np.array([[0,1],[1,0]]))
pauli_y = sparse.csr_matrix(np.array([[0,-1j],[1j,0]]))
pauli_z = sparse.csr_matrix(np.array([[1,0],[0,-1]]))
I = sparse.csr_matrix(np.eye(2))

    
def generate_op_at_site(L,site,operator):
    op = I
    for i in range(L):
        if i==site:
            current_op=operator
        else:
            current_op=I
        op = sparse.kron(op,current_op, format = "csr") if i!= 0 else current_op
    return op

def generate_paulis(L):
    X=[]
    Y=[]
    Z=[]
    for i in range(L): 
        X.append(generate_op_at_site(L,i,pauli_x))
        Y.append(generate_op_at_site(L,i,pauli_y))
        Z.append(generate_op_at_site(L,i,pauli_z))
    return X,Y,Z
    
#ZZX convention
def generate_TFI(L,h,X,Y,Z):
    H=0
    for i in range(L):
        H+=-Z[i]@Z[int((i+1)%L)]-h*X[i]
    return H
#XXZ convention
def generate_TFI_XXZ(L,h,X,Y,Z):
    H=0
    for i in range(L):
        H+=-X[i]@X[int((i+1)%L)]-h*Z[i]
    return H

def generate_P2(L,J,p,s,X,Y,Z):
    H=0
    for i in range(L):
        H+=2*J*(sparse.csr_matrix(np.eye(2**L))-Z[i]@Z[int((i+1)%L)])+p*s/2*(sparse.csr_matrix(np.eye(2**L))-X[i])
    return H

def generate_forward_backward(L):
    X,Y,Z = generate_paulis(L)
    Zf = []
    Xf = []
    Yf = []
        
    Zb = []
    Xb = []
    Yb = []

    I_single = sparse.csr_matrix(np.eye(2**L))
    for i in range(L):
        Xf.append(sparse.kron(I_single,X[i], format = "csr"))
        Yf.append(sparse.kron(I_single,Y[i], format = "csr"))
        Zf.append(sparse.kron(I_single,Z[i], format = "csr"))
        Xb.append(sparse.kron(X[i],I_single, format = "csr"))
        Yb.append(sparse.kron(Y[i],I_single, format = "csr"))
        Zb.append(sparse.kron(Z[i],I_single, format = "csr"))
    return Xf,Yf,Zf,Xb,Yb,Zb

def generate_full_P2(L,J,U,p,s,Xf,Yf,Zf,Xb,Yb,Zb):
    P2=0
    I_double=sparse.csr_matrix(np.eye(2**(2*L)))
    for i in range(L-1):
        P2+= 2*U*(I_double-Xf[i]@Xb[i])+2*J*(I_double-Zf[i]@Zb[i]@Zf[int((i+1)%L)]@Zb[int((i+1)%L)])+p*(s-2)/4*((s+2)/(s-2)*I_double-Xf[i]@Xb[i])-p*s/4*(Xf[i]+Xb[i])
    return P2
def generate_P2_2(L,J,p,s,X,Y,Z):
    H=0
    for i in range(L-1):
        H+=-2*J*(Z[i]@Z[int((i+1))])
    for i in range(L):
        H+=-p*s/2*X[i]
    return H
def generate_full_diss(L,J,U,g,t,Xf,Yf,Zf,Xb,Yb,Zb,Spf,Smf,Spb,Smb):
    P2=0
    I=sparse.csr_matrix(np.eye(2**(2*L)))
    for i in range(L):
        P2+= 2*U*(I-Zf[i]@Zb[i])+2*J*(I-Xf[i]@Xb[i]@Xf[int((i+1)%L)]@Xb[int((i+1)%L)])-g*t*Spf[i]@Spf[(i+1)%L]@Spb[i]@Spf[(i+1)%L]-t*(I-np.sqrt(1-p)*Smf[i]@Smf[(i+1)%L]@Spf[i]@Spf[(i+1)%L])@(I-np.sqrt(1-p)*Smb[i]@Smb[(i+1)%L]@Spb[i]@Spb[(i+1)%L])+t*I
    return P2
def get_adjacent_sites(L):
    adjacent = []
    for row in range(L):
        for col in range(L):
            current = row * L + col
            # Right neighbor (with periodic wrap)
            right = row * L + (col + 1) % L
            adjacent.append((current, right))
            # Down neighbor (with periodic wrap)
            down = ((row + 1) % L) * L + col
            adjacent.append((current, down))
    return adjacent
    
def generate_P2_diss_2d(L,J,g,t,X,Y,Z,Sp,Sm):
    H=0
    I = sparse.csr_matrix(np.eye(2**L))
    pairs = get_adjacent_sites(int(np.sqrt(L)))
    for pair in pairs:
        H+=2*J*(I-X[pair[0]]@X[pair[1]])
        H+=-g*(2*Sp[pair[0]]@Sp[pair[1]]-1/2*(I+Z[pair[0]])@(I+Z[pair[1]]))
    return H

def generate_P2_diss_1d(L,J,g,t,X,Y,Z,Sp,Sm):
    H=0
    I = sparse.csr_matrix(np.eye(2**L))
    pairs = get_adjacent_sites(int(np.sqrt(L)))
    for i in range(L):
        H+=2*J*(I-X[i]@X[(i+1)%L])
        H+=-t*(g*Sp[i]@Sp[(i+1)%L]+(I-g*Sm[i]@Sm[(i+1)%L]@Sp[i]@Sp[(i+1)%L])+I)
    return H


def generate_P2_adapt(L,J,g,X,Y,Z,Sp,Sm):
    H=0
    I = sparse.csr_matrix(np.eye(2**L))
    for i in range(L):
        H+=-2*J*(X[i]@X[(i+1)%L])-g/4*(2*Z[i]-Z[i]@Z[(i+1)%L]+X[i]@X[(i+1)%L]-Y[i]@Y[(i+1)%L]-1j*X[i]@Y[(i+1)%L]-1j*Y[i]@X[(i+1)%L])-(I-(1-np.sqrt(1-g))*Smf[i]@Smf[(i+1)%L]@Spf[i]@Spf[(i+1)%L])@(I-(1-np.sqrt(1-g))*Smb[i]@Smb[(i+1)%L]@Spb[i]@Spb[(i+1)%L])
    return H
    

def get_TFI_cor(L,h_array):
    X,Y,Z = generate_paulis(L)
    cor = []
    for h in h_array:
        H = generate_TFI(L,h,X,Y,Z)
        w,v = sparse.linalg.eigsh(H,k=2,which = "SA")
        cor.append(v[:,0].transpose()@Z[0]@Z[int(L/2)]@v[:,0]/np.linalg.norm(v[:,0])**2)
    return np.array(cor)
    

In [4]:
N=50
Ls=np.array([6])
hs = np.linspace(0,2,N)
fid_cor=np.zeros((4,N))

trace = np.zeros((4,N))
for i,L in enumerate(Ls):
    print(L)
    X,Y,Z = generate_paulis(L)
    for j,h in enumerate(hs):   
        print(h)
        H=generate_TFI_XXZ(L,h,X,Y,Z)
        w,state=sc.sparse.linalg.eigsh(H,k=1,which = "SA")

        trace[i,j]=np.sum(np.abs(state))
        sqrt_state = np.sqrt(np.abs(state[:,0]))
        density = sc.sparse.diags(np.abs(state[:,0]),0)
        density_trans = (X[0]@X[int(L/2)]@density@X[int(L/2)]@X[0])
        sigma_rho = (density@density_trans).diagonal()
        
        #sigma_sqrt = sc.sparse.diags(np.sqrt(density_trans))
        fid=(np.sqrt(sigma_rho)).sum()/trace[i,j]
        fid_cor[i,j]=fid
        print(fid)
    print()

6
0.0
0.9999999999999999
0.04081632653061224
0.9998958331629031
0.08163265306122448
0.999582800946224
0.12244897959183673
0.9990592404765066
0.16326530612244897
0.9983221548981269
0.2040816326530612
0.9973668769694976
0.24489795918367346
0.9961866019100483
0.2857142857142857
0.9947717945922777
0.32653061224489793
0.993109480225933
0.36734693877551017
0.9911824349857226
0.4081632653061224
0.9889683056416241
0.44897959183673464
0.9864387077145208
0.4897959183673469
0.9835583826062172
0.5306122448979591
0.9802845371117126
0.5714285714285714
0.9765665421585955
0.6122448979591836
0.9723462236050785
0.6530612244897959
0.9675590183446969
0.6938775510204082
0.9621362633637304
0.7346938775510203
0.9560087950267508
0.7755102040816326
0.9491118292136808
0.8163265306122448
0.94139077367643
0.8571428571428571
0.9328072596279047
0.8979591836734693
0.9233444040588147
0.9387755102040816
0.9130102794169483
0.9795918367346939
0.9018388526706523
1.0204081632653061
0.8898881965093765
1.0612244897959182
0.

In [18]:
N=50
Ls=np.array([4,8,12])
hs = np.linspace(0,2,N)
ren2_cor=np.zeros((4,N))
ren4_cor=np.zeros((4,N))
ren6_cor=np.zeros((4,N))


trace = np.zeros((4,N))
for i,L in enumerate(Ls):
    print(L)
    X,Y,Z = generate_paulis(L)
    for j,h in enumerate(hs):   
        print(h)
        H=generate_TFI_XXZ(L,h,X,Y,Z)
        w,state=sc.sparse.linalg.eigsh(H,k=1,which = "SA")

        trace[i,j]=np.sum(np.abs(state))
        #sqrt_state = np.sqrt(np.abs(state[:,0]))
        density = sc.sparse.diags_array(np.abs(state[:,0]))
        density_trans = (X[0]@X[int(L/2)]@density@X[int(L/2)]@X[0])
        sigma_rho = (density@density_trans)
        #sigma_sqrt = sc.sparse.diags(np.sqrt(density_trans))
        ren2=sigma_rho.trace()/(density@density).trace()
        ren2_cor[i,j]=ren2
        ren4=(sigma_rho@sigma_rho).trace()/(density@density@density@density).trace()
        ren4_cor[i,j]=ren4

        ren6=(sigma_rho@sigma_rho@sigma_rho).trace()/(density@density@density@density@density@density).trace()
        ren6_cor[i,j]=ren6
    print()

4
0.0
0.04081632653061224
0.08163265306122448
0.12244897959183673
0.16326530612244897
0.2040816326530612
0.24489795918367346
0.2857142857142857
0.32653061224489793
0.36734693877551017
0.4081632653061224
0.44897959183673464
0.4897959183673469
0.5306122448979591
0.5714285714285714
0.6122448979591836
0.6530612244897959
0.6938775510204082
0.7346938775510203
0.7755102040816326
0.8163265306122448
0.8571428571428571
0.8979591836734693
0.9387755102040816
0.9795918367346939
1.0204081632653061
1.0612244897959182
1.1020408163265305
1.1428571428571428
1.183673469387755
1.2244897959183672
1.2653061224489794
1.3061224489795917
1.346938775510204
1.3877551020408163
1.4285714285714284
1.4693877551020407
1.510204081632653
1.5510204081632653
1.5918367346938773
1.6326530612244896
1.673469387755102
1.7142857142857142
1.7551020408163265
1.7959183673469385
1.8367346938775508
1.8775510204081631
1.9183673469387754
1.9591836734693877
2.0

8
0.0
0.04081632653061224
0.08163265306122448
0.12244897959183673
0.16326

In [ ]:
L_index=7
field_names=["L","J","U","s"]
fields=[Ls[Li],J,U]
file_name="..\\data\\correlators_Z2\\Ising_exact_L"+str(Ls[L_index])+".csv"
with open(file_name, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(field_names)
    writer.writerow(fields)
    

data = {
"fid": fid_cor[L_index,:],  
"R2": R2[L_index,:],
"R4": R4[L_index,:],
"R6": R6[L_index,:],
"R8": R8[L_index,:],
"h": hs,
}

df = pd.DataFrame(data)

df.to_csv(file_name, mode='a', index=False, header=True)